In [ ]:
# -*- coding: utf-8 -*-
"""
Projeto: Classificação de Incêndios Florestais
Autor: Alysson Patricio

Versão adaptada a partir de um desafio prático da Fábrica de Software - UNIPÊ.
Objetivo: Construir e avaliar modelos de Machine Learning para prever o nível de alerta
com base em variáveis categóricas e numéricas.
"""

# ================================
# 1. Importando bibliotecas
# ================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, confusion_matrix
import joblib

# ================================
# 2. Carregando os dados
# ================================
dados = pd.read_csv("/content/Forest Fires.csv")

print("Amostra inicial da base:")
print(dados.head())

print("\nResumo da estrutura do DataFrame:")
dados.info()

# ================================
# 3. Tratamento de dados
# ================================
# Padronizando a coluna de alertas
dados['alertlevel'] = dados['alertlevel'].str.upper()

# Removendo categorias indesejadas
dados = dados[dados['alertlevel'] != 'ORANGE']

# Criando variável binária baseada na severidade
limiar_severidade = dados['severity'].median()
dados['nivel_alerta'] = dados['severity'].apply(
    lambda valor: 'ALTO' if valor > limiar_severidade else 'BAIXO'
)

print(f"\nMediana usada como limiar: {limiar_severidade}")
print("Distribuição da variável alvo binária:")
print(dados['nivel_alerta'].value_counts())

# ================================
# 4. Seleção de features
# ================================
atributos = ['country', 'severity', 'Duration (days)']
alvo = 'nivel_alerta'
base_modelo = dados[atributos + [alvo]]

X_dados = base_modelo.drop(columns=[alvo]).values
y_dados = base_modelo[alvo].values

print("\nDimensões:")
print("X_dados:", X_dados.shape)
print("y_dados:", y_dados.shape)

# ================================
# 5. Pré-processamento
# ================================
# Codificação + escalonamento
indice_categorico = 0  # coluna "country"

transformador_cat = ColumnTransformer(
    transformers=[('OneHot', OneHotEncoder(handle_unknown='ignore'), [indice_categorico])],
    remainder='passthrough'
)

pipeline = Pipeline(steps=[
    ('Codificação', transformador_cat),
    ('Normalização', StandardScaler(with_mean=False))
])

X_tratado = pipeline.fit_transform(X_dados)

# Codificação do alvo
encoder_alvo = LabelEncoder()
y_tratado = encoder_alvo.fit_transform(y_dados)
nomes_classes = encoder_alvo.classes_

print("\nDimensões após pré-processamento:")
print("X_tratado:", X_tratado.shape)
print("Classes do alvo:", nomes_classes)

# ================================
# 6. Divisão treino/teste
# ================================
X_train, X_test, y_train, y_test = train_test_split(
    X_tratado, y_tratado, test_size=0.2, random_state=42
)

print("\nDimensões dos conjuntos:")
print("Treino:", X_train.shape)
print("Teste:", X_test.shape)

# ================================
# 7. Treinamento de modelos
# ================================
modelo_logistico = LogisticRegression(max_iter=1000)
modelo_logistico.fit(X_train, y_train)
print("Modelo Regressão Logística treinado.")

modelo_nb = GaussianNB()
modelo_nb.fit(X_train.toarray(), y_train)
print("Modelo Naive Bayes treinado.")

modelo_arvore = DecisionTreeClassifier(random_state=42)
modelo_arvore.fit(X_train, y_train)
print("Modelo Árvore de Decisão treinado.")

modelo_rf = RandomForestClassifier(random_state=42)
modelo_rf.fit(X_train, y_train)
print("Modelo Random Forest treinado.")

# ================================
# 8. Avaliação
# ================================
def avaliar(y_real, y_prev, classes, nome_modelo):
    acuracia = accuracy_score(y_real, y_prev)
    matriz = confusion_matrix(y_real, y_prev)

    print(f"\n--- Avaliação do modelo: {nome_modelo} ---")
    print(f"Acurácia: {acuracia * 100:.2f}%")

    plt.figure(figsize=(5, 4))
    sns.heatmap(matriz, annot=True, fmt="d", cmap="Blues",
                xticklabels=classes, yticklabels=classes)
    plt.title(f"Matriz de Confusão - {nome_modelo}")
    plt.ylabel("Valor Real")
    plt.xlabel("Valor Predito")
    plt.show()

# Avaliando todos os modelos
avaliar(y_test, modelo_logistico.predict(X_test), nomes_classes, "Regressão Logística")
avaliar(y_test, modelo_nb.predict(X_test.toarray()), nomes_classes, "Naive Bayes")
avaliar(y_test, modelo_arvore.predict(X_test), nomes_classes, "Árvore de Decisão")
avaliar(y_test, modelo_rf.predict(X_test), nomes_classes, "Random Forest")

# ================================
# 9. Salvando o melhor modelo
# ================================
melhor_modelo = modelo_arvore

joblib.dump(melhor_modelo, "modelo_arvore_decisao.pkl")
joblib.dump(pipeline, "pipeline_preprocessamento.pkl")

print("\nModelo final (Árvore de Decisão) e pipeline de pré-processamento salvos